A copy from https://github.com/Heumi/BEGAN-tensorflow/tree/master/src/function

To test the difference

http://www.foldl.me/uploads/2015/conditional-gans-face-generation/paper.pdf

https://github.com/nightrome/really-awesome-gan

https://arxiv.org/pdf/1705.09966.pdf

https://arxiv.org/pdf/1703.05192.pdf

https://arxiv.org/pdf/1606.03657.pdf

https://arxiv.org/pdf/1704.02166.pdf

https://arxiv.org/pdf/1709.03842.pdf

In [1]:
import os
import scipy.misc as scm

def make_project_dir(project_dir):
    if not os.path.exists(project_dir):
        os.makedirs(project_dir)
        os.makedirs(os.path.join(project_dir, 'models'))
        os.makedirs(os.path.join(project_dir, 'result'))
        os.makedirs(os.path.join(project_dir, 'result_test'))


def get_image(img_path):
    img = scm.imread(img_path)/255. - 0.5
    img = img[..., ::-1]  # rgb to bgr
    return img


def inverse_image(img):
    img = (img + 0.5) * 255.
    img[img > 255] = 255
    img[img < 0] = 0
    img = img[..., ::-1] # bgr to rgb
    return img


In [2]:
import tensorflow as tf
import numpy as np


def conv2d(x, filter_shape, bias=True, stride=1, padding="SAME", name="conv2d"):
    kw, kh, nin, nout = filter_shape
    pad_size = (kw - 1) / 2

    if padding == "VALID":
        x = tf.pad(x, [[0, 0], [pad_size, pad_size], [pad_size, pad_size], [0, 0]], "SYMMETRIC")

    initializer = tf.random_normal_initializer(0., 0.02)
    with tf.variable_scope(name):
        weight = tf.get_variable("weight", shape=filter_shape, initializer=initializer)
        x = tf.nn.conv2d(x, weight, [1, stride, stride, 1], padding=padding)

        if bias:
            b = tf.get_variable("bias", shape=filter_shape[-1], initializer=tf.constant_initializer(0.))
            x = tf.nn.bias_add(x, b)
    return x


def fc(x, output_shape, bias=True, name='fc'):
    shape = x.get_shape().as_list()
    dim = np.prod(shape[1:])
    x = tf.reshape(x, [-1, dim])
    input_shape = dim

    initializer = tf.random_normal_initializer(0., 0.02)
    with tf.variable_scope(name):
        weight = tf.get_variable("weight", shape=[input_shape, output_shape], initializer=initializer)
        x = tf.matmul(x, weight)

        if bias:
            b = tf.get_variable("bias", shape=[output_shape], initializer=tf.constant_initializer(0.))
            x = tf.nn.bias_add(x, b)
    return x


def pool(x, r=2, s=1):
    return tf.nn.avg_pool(x, ksize=[1, r, r, 1], strides=[1, s, s, 1], padding="SAME")


def l1_loss(x, y):
    return tf.reduce_mean(tf.abs(x - y))


def resize_nn(x, size):
    return tf.image.resize_nearest_neighbor(x, size=(int(size), int(size)))

In [3]:
import glob
import time
import numpy as np
import tensorflow as tf

class op_base:
    def __init__(self, sess, project_name):
        self.sess = sess

        # Train
        self.flag = True #args.flag
        self.gpu_number = 0 #args.gpu_number
        self.project = project_name #"test_began" #args.project

        # Train Data
        self.data_dir = "./Face_data/Celeba/dataset_64x64" #args.data_dir #./Data
        self.dataset = "celeba_train_expr_test" #args.dataset  # celeba
        self.data_size = 64 #args.data_size  # 64 or 128
        self.data_opt = "crop" #args.data_opt  # raw or crop

        # Train Iteration
        self.niter = 500 #50 #args.niter
        self.niter_snapshot = 2440 #args.nsnapshot
        self.max_to_keep = 5 #args.max_to_keep

        # Train Parameter
        self.batch_size = 16 #args.batch_size
        self.learning_rate = 1e-4 #args.learning_rate
        self.mm = 0.5 #args.momentum
        self.mm2 = 0.999 #args.momentum2
        self.lamda = 0.001 #args.lamda
        self.gamma = 0.5 #args.gamma
        self.filter_number = 64 #args.filter_number
        self.input_size = 64 #args.input_size
        self.embedding = 64 #args.embedding

        # Result Dir & File
        self.project_dir = 'assets/{0}_{1}_{2}_{3}/'.format(self.project, self.dataset, self.data_opt, self.data_size)
        self.ckpt_dir = os.path.join(self.project_dir, 'models')
        self.model_name = "{0}.model".format(self.project)
        self.ckpt_model_name = os.path.join(self.ckpt_dir, self.model_name)

        # etc.
        if not os.path.exists('assets'):
            os.makedirs('assets')
        make_project_dir(self.project_dir)

    def load(self, sess, saver, ckpt_dir):
        ckpt = tf.train.get_checkpoint_state(ckpt_dir)
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        saver.restore(sess, os.path.join(ckpt_dir, ckpt_name))

In [4]:
import glob
import time
import datetime


class Operator(op_base):
    def __init__(self, sess, project_name):
        op_base.__init__(self, sess, project_name)
        self.build_model()

    def build_model(self):
        # Input placeholder
        self.x = tf.placeholder(tf.float32, shape=[None, self.data_size, self.data_size, 3], name='x')
        self.y = tf.placeholder(tf.float32, shape=[None, self.data_size, self.data_size, 3], name='y')
        self.lr = tf.placeholder(tf.float32, name='lr')

        # AE
        self.ae = self.decoder(self.encoder(self.x))
        self.ae_test = self.decoder(self.encoder(self.x, reuse=True), reuse=True)

        # Loss
        self.ae_loss = l1_loss(self.y, self.ae)
        self.test_loss = l1_loss(self.y, self.ae_test)

        # Variables
        ae_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, "ae_")

        # Optimizer
        self.opt_ae = tf.train.AdamOptimizer(self.lr, self.mm).minimize(self.ae_loss, var_list=ae_vars)

        # initializer
        self.sess.run(tf.global_variables_initializer())

        # tf saver
        self.saver = tf.train.Saver(max_to_keep=(self.max_to_keep))

        try:
            self.load(self.sess, self.saver, self.ckpt_dir)
        except:
            # save full graph
            self.saver.save(self.sess, self.ckpt_model_name, write_meta_graph=True)

        # Summary
        if self.flag:
            tf.summary.scalar('loss/loss', self.ae_loss)
            tf.summary.scalar('loss/test_loss', self.test_loss)
            self.merged = tf.summary.merge_all()
            self.writer = tf.summary.FileWriter(self.project_dir, self.sess.graph)

    def train(self, train_flag):
        # load data
        data_path = self.data_dir
        test_data_path = "./Face_data/Faces_with_expression_label/dataset_64x64" # expressions

        if os.path.exists(data_path + '.npy'):
            data = np.load(data_path + '.npy')
        else:
            data = sorted(glob.glob(os.path.join(data_path, "*.*")))
            np.save(data_path + '.npy', data)
            
        if os.path.exists(test_data_path + '.npy'):
            test_data = np.load(test_data_path + '.npy')
        else:
            test_data = sorted(glob.glob(os.path.join(test_data_path, "*.*")))
            np.save(test_data_path + '.npy', test_data)

        print('Shuffle ....')
        random_order = np.random.permutation(len(data))
        data = [data[i] for i in random_order[:]]
        print('Shuffle Done')

        # initial parameter
        start_time = time.time()
        lr = np.float32(self.learning_rate)
        self.count = 0

        for epoch in range(self.niter):
            batch_idxs = len(data) // self.batch_size
            
            # Get a random batch of test data for the whole epoch
            test_random_order = np.random.permutation(len(test_data))
            test_data = [test_data[i] for i in test_random_order[:]]
            test_files = test_data[0:self.batch_size]
            test_data_batch = [get_image(test_file[0]) for test_file in test_files]

            for idx in range(0, batch_idxs):
                self.count += 1

                batch_files = data[idx * self.batch_size: (idx + 1) * self.batch_size]
                batch_data = [get_image(batch_file) for batch_file in batch_files]

                # opt & feed list 
                ae_opt = [self.opt_ae, self.ae_loss]
                feed_dict = {self.x: batch_data, self.y: batch_data, self.lr: lr}

                # run tensorflow
                _, loss_ae = self.sess.run(ae_opt, feed_dict=feed_dict)
                
                # test & feed list 
                ae_test = [self.test_loss, self.ae_test, self.merged]
                feed_dict = {self.x: test_data_batch, self.y: test_data_batch}

                # run tensorflow
                test_loss, _, summary = self.sess.run(ae_test, feed_dict=feed_dict)
        
                if self.count % 100 == 1:
                    print("Epoch: [%2d] [%4d/%4d] time: %4.4f, "
                          "loss: %.4f, test_loss: %.4f"
                          % (epoch, idx, batch_idxs, time.time() - start_time,
                             loss_ae, test_loss))

                # write train summary
                self.writer.add_summary(summary, self.count)

                # Test during Training
                if self.count % self.niter_snapshot == (self.niter_snapshot - 1):
                    # save & test
                    self.saver.save(self.sess, self.ckpt_model_name, global_step=self.count, write_meta_graph=False)
                    self.test(train_flag)

    def test(self, train_flag=True):
        print('Test Sample Generation...')
        # generate output
        img_num = 18
        img_size = self.data_size
        
        # load data test
        data_path = "./Face_data/Faces_with_expression_label/dataset_64x64" # expressions

        if os.path.exists(data_path + '.npy'):
            data = np.load(data_path + '.npy')
        else:
            data = sorted(glob.glob(os.path.join(data_path, "*.*")))
            np.save(data_path + '.npy', data)

        # shuffle test data
        random_order = np.random.permutation(len(data))
        data = [data[i] for i in random_order[:]]

        output_f = int(np.sqrt(img_num * 2))
        im_output_gen = np.zeros([img_size * output_f, img_size * output_f, 3])

        test_files = data[0: img_num]
        test_data = [get_image(test_file[0]) for test_file in test_files]
        test_data_o = [scm.imread(test_file[0]) for test_file in test_files]
        
        
        output_gen = (self.sess.run(self.ae_test, feed_dict={self.x: test_data}))  # generator output

        output_gen = [inverse_image(output_gen[i]) for i in range(img_num)]

        for i in range(output_f):
            for j in range(output_f):
                if j % 2 == 0:
                    im_output_gen[i * img_size:(i + 1) * img_size, j * img_size:(j + 1) * img_size, :] \
                        = test_data_o[int(j/2) + (i * int(output_f/2))]
                elif j % 2 == 1:
                    im_output_gen[i * img_size:(i + 1) * img_size, j * img_size:(j + 1) * img_size, :] \
                        = output_gen[int(j/2) + (i * int(output_f/2))]

        # output save
        scm.imsave(self.project_dir + '/result/' + str(self.count) + '_output.bmp', im_output_gen)
       

In [5]:

class AE(Operator):
    def __init__(self, sess, project_name):
        Operator.__init__(self, sess, project_name)


    def encoder(self, x, reuse=None):
        with tf.variable_scope('ae_') as scope:
            if reuse:
                scope.reuse_variables()

            f = self.filter_number
            h = self.embedding
            p = "SAME"

            x = conv2d(x, [3, 3, 3, f], stride=1,  padding=p,name='conv1_enc_a')
            x = tf.nn.elu(x)

            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p,name='conv2_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1,  padding=p,name='conv2_enc_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [1, 1, f, 2 * f], stride=1,  padding=p,name='conv3_enc_0')
            x = pool(x, r=2, s=2)
            x = conv2d(x, [3, 3, 2 * f, 2 * f], stride=1,  padding=p,name='conv3_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, 2 * f, 2 * f], stride=1,  padding=p,name='conv3_enc_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [1, 1, 2 * f, 3 * f], stride=1,  padding=p,name='conv4_enc_0')
            x = pool(x, r=2, s=2)
            x = conv2d(x, [3, 3, 3 * f, 3 * f], stride=1,  padding=p,name='conv4_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, 3 * f, 3 * f], stride=1,  padding=p,name='conv4_enc_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [1, 1, 3 * f, 4 * f], stride=1,  padding=p,name='conv5_enc_0')
            x = pool(x, r=2, s=2)
            x = conv2d(x, [3, 3, 4 * f, 4 * f], stride=1,  padding=p,name='conv5_enc_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, 4 * f, 4 * f], stride=1,  padding=p,name='conv5_enc_b')
            x = tf.nn.elu(x)

            if self.data_size == 128:
                x = conv2d(x, [1, 1, 4 * f, 5 * f], stride=1,  padding=p,name='conv6_enc_0')
                x = pool(x, r=2, s=2)
                x = conv2d(x, [3, 3, 5 * f, 5 * f], stride=1,  padding=p,name='conv6_enc_a')
                x = tf.nn.elu(x)
                x = conv2d(x, [3, 3, 5 * f, 5 * f], stride=1,  padding=p,name='conv6_enc_b')
                x = tf.nn.elu(x)

            x = fc(x, h, name='enc_fc')
        return x

    def decoder(self, x, reuse=None):
        with tf.variable_scope('ae_') as scope:
            if reuse:
                scope.reuse_variables()

            w = self.data_size
            f = self.filter_number
            p = "SAME"

            x = fc(x, 8 * 8 * f, name='fc')
            x = tf.reshape(x, [-1, 8, 8, f])

            x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv1_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv1_b')
            x = tf.nn.elu(x)

            if self.data_size == 128:
                x = resize_nn(x, w / 8)
                x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv2_a')
                x = tf.nn.elu(x)
                x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv2_b')
                x = tf.nn.elu(x)

                x = resize_nn(x, w / 4)
            x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv3_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv3_b')
            x = tf.nn.elu(x)

            x = resize_nn(x, w / 2)
            x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv4_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv4_b')
            x = tf.nn.elu(x)

            x = resize_nn(x, w)
            x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv5_a')
            x = tf.nn.elu(x)
            x = conv2d(x, [3, 3, f, f], stride=1, padding=p, name='conv5_b')
            x = tf.nn.elu(x)

            x = conv2d(x, [3, 3, f, 3], stride=1, padding=p, name='conv6_a')
        return x

In [6]:
import distutils.util
import os
import tensorflow as tf

''' config settings '''

project_name = "AE_Face_3"
train_flag = True

'''-----------------'''

gpu_number = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #args.gpu_number

with tf.device('/gpu:{0}'.format(gpu_number)):
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.90)
    config = tf.ConfigProto(allow_soft_placement=True, gpu_options=gpu_options)

    with tf.Session(config=config) as sess:
        model = AE(sess, project_name)

        # TRAIN / TEST
        if train_flag:
            model.train(train_flag)
        else:
            model.test(train_flag)

Shuffle ....
Shuffle Done
Epoch: [ 0] [   0/12080] time: 0.7380, loss: 0.2280, test_loss: 0.2458
Epoch: [ 0] [ 100/12080] time: 5.9544, loss: 0.1327, test_loss: 0.1456
Epoch: [ 0] [ 200/12080] time: 11.2149, loss: 0.1155, test_loss: 0.1083
Epoch: [ 0] [ 300/12080] time: 16.4034, loss: 0.1087, test_loss: 0.0960
Epoch: [ 0] [ 400/12080] time: 21.4602, loss: 0.0981, test_loss: 0.0916
Epoch: [ 0] [ 500/12080] time: 26.5706, loss: 0.0954, test_loss: 0.0939
Epoch: [ 0] [ 600/12080] time: 31.7939, loss: 0.0962, test_loss: 0.0930
Epoch: [ 0] [ 700/12080] time: 36.9862, loss: 0.0964, test_loss: 0.0895
Epoch: [ 0] [ 800/12080] time: 42.0825, loss: 0.0877, test_loss: 0.0920
Epoch: [ 0] [ 900/12080] time: 47.2338, loss: 0.0953, test_loss: 0.0824
Epoch: [ 0] [1000/12080] time: 52.3578, loss: 0.0741, test_loss: 0.0804
Epoch: [ 0] [1100/12080] time: 57.6187, loss: 0.0833, test_loss: 0.0864
Epoch: [ 0] [1200/12080] time: 62.9325, loss: 0.0809, test_loss: 0.0747
Epoch: [ 0] [1300/12080] time: 68.2766, 

Epoch: [ 0] [11100/12080] time: 596.6129, loss: 0.0599, test_loss: 0.0514
Epoch: [ 0] [11200/12080] time: 601.9516, loss: 0.0525, test_loss: 0.0499
Epoch: [ 0] [11300/12080] time: 607.3123, loss: 0.0564, test_loss: 0.0496
Epoch: [ 0] [11400/12080] time: 612.6649, loss: 0.0700, test_loss: 0.0509
Epoch: [ 0] [11500/12080] time: 618.0128, loss: 0.0637, test_loss: 0.0502
Epoch: [ 0] [11600/12080] time: 623.3250, loss: 0.0613, test_loss: 0.0515
Epoch: [ 0] [11700/12080] time: 628.6932, loss: 0.0649, test_loss: 0.0516
Epoch: [ 0] [11800/12080] time: 634.1707, loss: 0.0572, test_loss: 0.0507
Epoch: [ 0] [11900/12080] time: 639.6371, loss: 0.0598, test_loss: 0.0518
Epoch: [ 0] [12000/12080] time: 644.9946, loss: 0.0613, test_loss: 0.0560
Epoch: [ 1] [  20/12080] time: 650.4486, loss: 0.0591, test_loss: 0.0485
Test Sample Generation...
Epoch: [ 1] [ 120/12080] time: 655.8643, loss: 0.0600, test_loss: 0.0505
Epoch: [ 1] [ 220/12080] time: 661.1939, loss: 0.0550, test_loss: 0.0482
Epoch: [ 1] [ 3

Epoch: [ 1] [10020/12080] time: 1188.3132, loss: 0.0558, test_loss: 0.0457
Epoch: [ 1] [10120/12080] time: 1193.8007, loss: 0.0513, test_loss: 0.0437
Epoch: [ 1] [10220/12080] time: 1199.1595, loss: 0.0595, test_loss: 0.0461
Epoch: [ 1] [10320/12080] time: 1204.5334, loss: 0.0543, test_loss: 0.0445
Epoch: [ 1] [10420/12080] time: 1209.8845, loss: 0.0555, test_loss: 0.0448
Epoch: [ 1] [10520/12080] time: 1215.2505, loss: 0.0593, test_loss: 0.0457
Epoch: [ 1] [10620/12080] time: 1220.5653, loss: 0.0500, test_loss: 0.0440
Epoch: [ 1] [10720/12080] time: 1225.8984, loss: 0.0544, test_loss: 0.0444
Epoch: [ 1] [10820/12080] time: 1231.3042, loss: 0.0586, test_loss: 0.0449
Epoch: [ 1] [10920/12080] time: 1236.6774, loss: 0.0544, test_loss: 0.0507
Epoch: [ 1] [11020/12080] time: 1242.0390, loss: 0.0491, test_loss: 0.0439
Epoch: [ 1] [11120/12080] time: 1247.4568, loss: 0.0500, test_loss: 0.0461
Epoch: [ 1] [11220/12080] time: 1252.7996, loss: 0.0610, test_loss: 0.0436
Epoch: [ 1] [11320/12080]

Epoch: [ 2] [8940/12080] time: 1779.9822, loss: 0.0494, test_loss: 0.0417
Epoch: [ 2] [9040/12080] time: 1785.4149, loss: 0.0512, test_loss: 0.0421
Epoch: [ 2] [9140/12080] time: 1790.8360, loss: 0.0585, test_loss: 0.0439
Epoch: [ 2] [9240/12080] time: 1796.2271, loss: 0.0571, test_loss: 0.0417
Epoch: [ 2] [9340/12080] time: 1801.6264, loss: 0.0549, test_loss: 0.0426
Epoch: [ 2] [9440/12080] time: 1806.9782, loss: 0.0561, test_loss: 0.0417
Epoch: [ 2] [9540/12080] time: 1812.3522, loss: 0.0579, test_loss: 0.0432
Epoch: [ 2] [9640/12080] time: 1817.7384, loss: 0.0499, test_loss: 0.0473
Epoch: [ 2] [9740/12080] time: 1823.1069, loss: 0.0486, test_loss: 0.0425
Epoch: [ 2] [9840/12080] time: 1828.4844, loss: 0.0498, test_loss: 0.0453
Epoch: [ 2] [9940/12080] time: 1833.8269, loss: 0.0549, test_loss: 0.0418
Test Sample Generation...
Epoch: [ 2] [10040/12080] time: 1839.2295, loss: 0.0583, test_loss: 0.0417
Epoch: [ 2] [10140/12080] time: 1844.6402, loss: 0.0501, test_loss: 0.0428
Epoch: [ 2

Epoch: [ 3] [7760/12080] time: 2367.1026, loss: 0.0587, test_loss: 0.0416
Epoch: [ 3] [7860/12080] time: 2372.4747, loss: 0.0504, test_loss: 0.0388
Epoch: [ 3] [7960/12080] time: 2377.8164, loss: 0.0537, test_loss: 0.0403
Epoch: [ 3] [8060/12080] time: 2383.2536, loss: 0.0507, test_loss: 0.0402
Epoch: [ 3] [8160/12080] time: 2388.5956, loss: 0.0553, test_loss: 0.0396
Epoch: [ 3] [8260/12080] time: 2394.0050, loss: 0.0518, test_loss: 0.0401
Epoch: [ 3] [8360/12080] time: 2399.4052, loss: 0.0626, test_loss: 0.0394
Epoch: [ 3] [8460/12080] time: 2404.8652, loss: 0.0531, test_loss: 0.0392
Epoch: [ 3] [8560/12080] time: 2410.2985, loss: 0.0530, test_loss: 0.0403
Epoch: [ 3] [8660/12080] time: 2415.6528, loss: 0.0541, test_loss: 0.0392
Epoch: [ 3] [8760/12080] time: 2421.0922, loss: 0.0514, test_loss: 0.0399
Epoch: [ 3] [8860/12080] time: 2426.3822, loss: 0.0603, test_loss: 0.0423
Epoch: [ 3] [8960/12080] time: 2431.7317, loss: 0.0482, test_loss: 0.0396
Epoch: [ 3] [9060/12080] time: 2437.10

...

Epoch: [ 8] [2160/12080] time: 5315.0654, loss: 0.0510, test_loss: 0.0391
Epoch: [ 8] [2260/12080] time: 5320.4733, loss: 0.0468, test_loss: 0.0386
Epoch: [ 8] [2360/12080] time: 5325.8491, loss: 0.0473, test_loss: 0.0394
Epoch: [ 8] [2460/12080] time: 5331.1912, loss: 0.0447, test_loss: 0.0383
Epoch: [ 8] [2560/12080] time: 5336.5531, loss: 0.0455, test_loss: 0.0388
Epoch: [ 8] [2660/12080] time: 5341.9360, loss: 0.0534, test_loss: 0.0385
Epoch: [ 8] [2760/12080] time: 5347.2778, loss: 0.0483, test_loss: 0.0390
Epoch: [ 8] [2860/12080] time: 5352.6703, loss: 0.0479, test_loss: 0.0393
Epoch: [ 8] [2960/12080] time: 5358.0864, loss: 0.0494, test_loss: 0.0392
Epoch: [ 8] [3060/12080] time: 5363.4718, loss: 0.0407, test_loss: 0.0392
Epoch: [ 8] [3160/12080] time: 5368.9126, loss: 0.0432, test_loss: 0.0393
Epoch: [ 8] [3260/12080] time: 5374.2674, loss: 0.0470, test_loss: 0.0385
Epoch: [ 8] [3360/12080] time: 5379.7585, loss: 0.0474, test_loss: 0.0386
Test Sample Generation...
Epoch: [ 8] 

Test Sample Generation...
Epoch: [ 9] [1080/12080] time: 5907.2230, loss: 0.0467, test_loss: 0.0360
Epoch: [ 9] [1180/12080] time: 5912.5813, loss: 0.0462, test_loss: 0.0365
Epoch: [ 9] [1280/12080] time: 5917.9118, loss: 0.0470, test_loss: 0.0358
Epoch: [ 9] [1380/12080] time: 5923.3320, loss: 0.0483, test_loss: 0.0359
Epoch: [ 9] [1480/12080] time: 5928.7395, loss: 0.0506, test_loss: 0.0370
Epoch: [ 9] [1580/12080] time: 5934.1529, loss: 0.0486, test_loss: 0.0358
Epoch: [ 9] [1680/12080] time: 5939.5303, loss: 0.0435, test_loss: 0.0373
Epoch: [ 9] [1780/12080] time: 5944.8532, loss: 0.0493, test_loss: 0.0366
Epoch: [ 9] [1880/12080] time: 5950.3592, loss: 0.0461, test_loss: 0.0371
Epoch: [ 9] [1980/12080] time: 5955.7485, loss: 0.0452, test_loss: 0.0366
Epoch: [ 9] [2080/12080] time: 5961.1290, loss: 0.0477, test_loss: 0.0373
Epoch: [ 9] [2180/12080] time: 5966.4951, loss: 0.0513, test_loss: 0.0357
Epoch: [ 9] [2280/12080] time: 5971.8423, loss: 0.0466, test_loss: 0.0358
Epoch: [ 9] 

Epoch: [ 9] [11980/12080] time: 6493.5110, loss: 0.0532, test_loss: 0.0367
Epoch: [10] [   0/12080] time: 6498.8530, loss: 0.0468, test_loss: 0.0386
Epoch: [10] [ 100/12080] time: 6504.2081, loss: 0.0513, test_loss: 0.0382
Epoch: [10] [ 200/12080] time: 6509.6090, loss: 0.0486, test_loss: 0.0384
Epoch: [10] [ 300/12080] time: 6514.9819, loss: 0.0515, test_loss: 0.0388
Epoch: [10] [ 400/12080] time: 6520.4670, loss: 0.0472, test_loss: 0.0383
Epoch: [10] [ 500/12080] time: 6525.8289, loss: 0.0455, test_loss: 0.0384
Epoch: [10] [ 600/12080] time: 6531.2647, loss: 0.0517, test_loss: 0.0380
Epoch: [10] [ 700/12080] time: 6536.6701, loss: 0.0517, test_loss: 0.0380
Epoch: [10] [ 800/12080] time: 6542.0425, loss: 0.0467, test_loss: 0.0386
Epoch: [10] [ 900/12080] time: 6547.4276, loss: 0.0533, test_loss: 0.0390
Epoch: [10] [1000/12080] time: 6552.7955, loss: 0.0463, test_loss: 0.0379
Epoch: [10] [1100/12080] time: 6558.1623, loss: 0.0484, test_loss: 0.0381
Test Sample Generation...
Epoch: [10]

Epoch: [10] [10900/12080] time: 7085.1038, loss: 0.0501, test_loss: 0.0379
Test Sample Generation...
Epoch: [10] [11000/12080] time: 7090.5479, loss: 0.0485, test_loss: 0.0388
Epoch: [10] [11100/12080] time: 7095.9182, loss: 0.0468, test_loss: 0.0382
Epoch: [10] [11200/12080] time: 7101.2576, loss: 0.0425, test_loss: 0.0389
Epoch: [10] [11300/12080] time: 7106.6221, loss: 0.0450, test_loss: 0.0380
Epoch: [10] [11400/12080] time: 7111.9978, loss: 0.0557, test_loss: 0.0388
Epoch: [10] [11500/12080] time: 7117.3466, loss: 0.0503, test_loss: 0.0381
Epoch: [10] [11600/12080] time: 7122.8077, loss: 0.0508, test_loss: 0.0397
Epoch: [10] [11700/12080] time: 7128.1579, loss: 0.0507, test_loss: 0.0380
Epoch: [10] [11800/12080] time: 7133.5519, loss: 0.0452, test_loss: 0.0381
Epoch: [10] [11900/12080] time: 7138.9158, loss: 0.0482, test_loss: 0.0380
Epoch: [10] [12000/12080] time: 7144.2428, loss: 0.0460, test_loss: 0.0378
Epoch: [11] [  20/12080] time: 7149.5981, loss: 0.0490, test_loss: 0.0375


KeyboardInterrupt: 